In [1]:
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
import cv2
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D,GlobalAveragePooling2D, Input

from keras.layers import Input, BatchNormalization, Activation
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.optimizers import Adam
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
#from mlxtend.plotting import plot_confusion_matrix
import random
from imblearn.over_sampling import RandomOverSampler
from generator import Generator

/home/lkgupta/my_project_dir/my_project_env/lib/python3.5/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/home/lkgupta/my_project_dir/my_project_env/lib/python3.5/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/lkgupta/my_project_dir/my_project_env/lib/python3.5/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print(tf.config.experimental.list_physical_devices('GPU'))
print(tf.__version__)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10533606760803238208
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 15207951283342419975
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 10010748388820436875
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 4912535035238137563
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:2"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 4058606711869938350
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:3"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 12108627298704196966
physical_device_desc: "device: XLA_GPU device"
]
[]
2.3.1


## Reading Data 

In [3]:
labelsdf= pd.read_csv("worDwiseLabels/gt.csv")

In [4]:
labelsdf.head()

,File_name,Script,translation,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,word_1.png,Arabic,فروجنا,NaN,NaN,NaN,NaN
1,word_2.png,Arabic,المشوى,NaN,NaN,NaN,NaN
2,word_3.png,Arabic,حلال,NaN,NaN,NaN,NaN
3,word_4.png,Arabic,و,NaN,NaN,NaN,NaN
4,word_5.png,Arabic,طازج,NaN,NaN,NaN,NaN


In [5]:
(labelsdf['Script']).value_counts()

Latin       42629
Korean       4476
Japanese     4108
Arabic       3505
Bangla       3214
Chinese      2702
Symbols      1133
3               1
Name: Script, dtype: int64

### Setting ninimum size of images
As the height and width of the input will decrease based on the values of kernel_size and strides. If the input image size is too small then we might fall short of the minimum required height and width (which should be greater than or equal to the kernel size) for the next convolution block.

In [16]:
def resize_image(img, min_side_len):

    h, w, c = img.shape

    # limit the min side maintaining the aspect ratio
    if min(h, w) < min_side_len:
        im_scale = float(min_side_len) / h if h < w else float(min_side_len) / w
    else:
        im_scale = 1.

    new_h = int(h * im_scale)
    new_w = int(w * im_scale)

    re_im = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
    return re_im  #, new_h / h, new_w / w

In [19]:
min_side_len = 64

directory = 'wordWise_fcn/'

if not os.path.exists(directory):
    os.makedirs(directory)
    
for filename in os.listdir('wordWise'):
    
    img = cv2.imread(os.path.join('wordWise',filename))
    new_im=resize_image(img, min_side_len)
        
    path = directory + filename
    cv2.imwrite(path, new_im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## Removing nas and unnecesary scripts

In [7]:

labelsdf = labelsdf.dropna(axis=0, subset=['Script'])
labelsdf = labelsdf.dropna(axis=0, subset=['File_name'])
labelsdf=labelsdf[(labelsdf['Script'] !='Symbols')]
labelsdf=labelsdf[(labelsdf['Script'] !='3')]

In [ ]:
mergeDF=pd.merge(labelsdf, imageInfoDf, left_on="File_name", right_on="fileName")
mergeDF.describe()

In [6]:
class_index={0:'Arabic',1:'Bangla', 2:'Chinese', 3:'Japanese', 4:'Korean',5:'Latin'}
#class_index={'Arabic': 0,'Bangla': 1,'Chinese': 2,'Japanese': 3,'Korean': 4,'Latin': 5}
total=labelsdf['Script'].value_counts().sum()
vc=labelsdf['Script'].value_counts()
class_weights={val:total/vc[class_index[val]]for val in class_index}
class_weights

{0: 17.622824536376605,
 1: 19.218419415059117,
 2: 22.860103626943005,
 3: 15.036027263875365,
 4: 13.79982126899017,
 5: 1.4489666658847264}

In [7]:
labelsdf_train_val = labelsdf.sample(frac = 0.85) 
labelsdf_test = labelsdf.drop(labelsdf_train_val.index)

#labelsdf_train_val = labelsdf_train_val.reset_index()

labelsdf_train = labelsdf_train_val.sample(frac = 0.85) 
labelsdf_val = labelsdf_train_val.drop(labelsdf_train.index)

labelsdf_train = labelsdf_train.reset_index()


In [8]:
(labelsdf_val['Script']).value_counts()

Latin       5457
Japanese     541
Korean       527
Arabic       448
Bangla       432
Chinese      336
Symbols      135
Name: Script, dtype: int64

In [9]:
(labelsdf_test['Script']).value_counts()

Latin       6467
Korean       646
Japanese     576
Arabic       534
Bangla       471
Chinese      394
Symbols      176
3              1
Name: Script, dtype: int64

In [10]:
(labelsdf_train['Script']).value_counts()

Latin       30705
Korean       3303
Japanese     2991
Arabic       2523
Bangla       2311
Chinese      1972
Symbols       822
Name: Script, dtype: int64

In [11]:
imageDir = 'wordWise_fcn/'
    
# If you get out of memory error try reducing the batch size
BATCH_SIZE=16

train_it = Generator( imageDir,labelsdf_train,'File_name','Script', BATCH_SIZE=16, shuffle_images=True, image_min_side=32)
val_it = Generator( imageDir,labelsdf_val,'File_name','Script', BATCH_SIZE=16, shuffle_images=True, image_min_side=32)
test_it = Generator( imageDir,labelsdf_test,'File_name','Script', BATCH_SIZE=16, shuffle_images=True, image_min_side=32)

In [14]:
def FCN_model(len_classes=6, dropout_rate=0.2):
    
    input = tf.keras.layers.Input(shape=(None, None, 3))

    x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1)(input)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)


    x = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)


    x = tf.keras.layers.Conv2D(filters=64, kernel_size=1, strides=1)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

 
    x = tf.keras.layers.Conv2D(filters=len_classes, kernel_size=1, strides=1)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.GlobalMaxPooling2D()(x)
    predictions = tf.keras.layers.Activation('softmax')(x)

    model = tf.keras.Model(inputs=input, outputs=predictions)
    
    print(model.summary())
    #print(f'Total number of layers: {len(model.layers)}')

    return model

In [15]:
strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Number of devices: 1


In [16]:
with strategy.scope():
    fcn_model=FCN_model()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv2d (Conv2D)              (None, None, None, 32)    896       
_________________________________________________________________
dropout (Dropout)            (None, None, None, 32)    0         
_________________________________________________________________
batch_normalization (BatchNo (None, None, None, 32)    128       
_________________________________________________________________
activation (Activation)      (None, None, None, 32)    0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 128)   36992     
_________________________________________________________________
dropout_1 (Dropout)          (None, None, None, 128)  

In [17]:
from keras.optimizers import SGD, Adam
adam = Adam(lr=0.001)
fcn_model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['Recall','Precision','Accuracy'])

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=4)
checkpoint = ModelCheckpoint('fcnModel.h5',save_best_only=True,monitor='val_loss')
historyInc = fcn_model.fit_generator(generator=train_it,steps_per_epoch=len(train_it), 
                              validation_data=val_it, validation_steps=len(val_it),
                              epochs=5,callbacks=[es,checkpoint])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
2738/2738 [==============================] - 9409s 3s/step - recall: 0.6713 - loss: 1.1080 - accuracy: 0.0000e+00 - precision: 0.7035 - val_accuracy: 0.0000e+00 - val_precision: 0.7113 - val_recall: 0.6965 - val_loss: 1.0830
Epoch 2/5
2738/2738 [==============================] - 11703s 4s/step - recall: 0.6939 - loss: 1.0641 - accuracy: 0.0000e+00 - precision: 0.7067 - val_accuracy: 0.0000e+00 - val_precision: 0.7822 - val_recall: 0.4609 - val_loss: 1.1375
Epoch 3/5
2738/2738 [==============================] - 15332s 6s/step - recall: 0.6920 - loss: 1.0431 - accuracy: 0.0000e+00 - precision: 0.7098 - val_accuracy: 0.0000e+00 - val_precision: 0.8138 - val_recall: 0.2687 - val_loss: 1.1812
Epoch 4/5
 986/2738 [=========>....................] - ETA: 1:27:04 - recall: 0.6883 - loss: 1.0317 - accuracy: 0.0000e+00 - precision:

In [12]:
fcn_model = keras.models.load_model("fcnModel.h5")

In [14]:
score = fcn_model.evaluate_generator(test_it)
print('Test loss:', score[0])
print('Test recall:', score[1])
#Confution Matrix and Classification Report
Y_predFCN = fcn_model.predict(test_it) 
Y_predFCN = np.argmax(Y_predFCN, axis=1)


Test loss: 1.057516098022461
Test recall: 0.7014744877815247
